In [2]:
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop, Adam, Adagrad
from matplotlib import pyplot as plt

In [9]:
with open('FormatedData/100000Packs/MetaData/context_words.pickle', 'rb') as file:
    context_words = pickle.load(file)
with open('FormatedData/100000Packs/MetaData/unique_tokens.pickle', 'rb') as file:
    unique_tokens = pickle.load(file)

In [3]:
# Metrics
def top_5_categorical_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)
def top_10_categorical_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=10)

def save_model(number, model):
    model.save('FinalModel/Model' + str(number) + '.h5')
    
def save_history(number, history):
    pickle.dump(history, open("FinalModel/history" + str(number) + ".p", "wb"))
    
def load_data(number):
    folder_path = f'FormatedData/100000Packs/{number}/'
    
    with open(folder_path + 'x_train.pickle', 'rb') as file:
        x_train = pickle.load(file)
    with open(folder_path + 'x_val.pickle', 'rb') as file:
        x_val = pickle.load(file)
    with open(folder_path + 'y_train.pickle', 'rb') as file:
        y_train = pickle.load(file)
    with open(folder_path + 'y_val.pickle', 'rb') as file:
        y_val = pickle.load(file)
    
    return x_train, x_val, y_train, y_val

In [4]:
dropout=.5
keras.backend.clear_session()
# Create Model
model = Sequential()
model.add(LSTM(128, input_shape=(context_words, len(unique_tokens)), return_sequences=True, recurrent_dropout=dropout,  dropout=dropout, activation="softsign"))
model.add(LSTM(128, recurrent_dropout=dropout, dropout=dropout, activation="softsign", return_sequences=True))
model.add(LSTM(128, recurrent_dropout=dropout, dropout=dropout, activation="softsign"))
model.add(Dense(len(unique_tokens), activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer=Adagrad(learning_rate=1), metrics=['accuracy', top_5_categorical_accuracy, top_10_categorical_accuracy])

In [ ]:
for j in range(1,6):
    # Train Model
    x_train, x_val, y_train, y_val = load_data(j)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=128, epochs=20, shuffle=True).history
    save_history(str(i)+"."+str(j),history)
save_model(1,model)

In [ ]:
# for i in range(1,6):
#     for j in range(1,6):
#         # Train Model
#         x_train, x_val, y_train, y_val = load_data(j)
#         history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=128, epochs=1, shuffle=True).history
#         save_history(str(i)+"."+str(j),history)
#     save_model(i,model)

743/743 [==============================] - 872s 1s/step - loss: 6.7300 - accuracy: 0.0695 - top_5_categorical_accuracy: 0.1801 - top_10_categorical_accuracy: 0.2509 - val_loss: 6.6916 - val_accuracy: 0.0850 - val_top_5_categorical_accuracy: 0.1802 - val_top_10_categorical_accuracy: 0.2728
